In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from scipy import stats
import statsmodels.api as sm
from datetime import date
from sklearn import metrics

In [2]:
# Load in the frame
ames = pd.read_csv('./data/train.csv')

In [3]:
len(ames.columns)

81

# Data Prep

## Summary Stats

In [4]:
ames[ames['Lot Frontage'].isna()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
7,145,535154050,20,RL,NaN,12160,Pave,NaN,IR1,Lvl,...,0,0,NaN,MnPrv,NaN,0,5,2010,COD,142000
8,1942,535353130,20,RL,NaN,15783,Pave,NaN,Reg,Lvl,...,0,0,NaN,MnPrv,Shed,400,6,2007,WD,112500
23,12,527165230,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,...,0,0,NaN,GdPrv,Shed,500,3,2010,WD,185000
27,1534,909277040,50,RL,NaN,11700,Pave,Grvl,IR1,HLS,...,0,0,NaN,NaN,NaN,0,7,2008,WD,198000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,560,532376110,20,RL,NaN,7791,Pave,NaN,IR1,Lvl,...,0,0,NaN,GdWo,NaN,0,10,2009,WD,129000
2022,2872,909475020,20,RL,NaN,16381,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,12,2006,WD,223000
2028,2526,534127190,20,RL,NaN,20781,Pave,NaN,IR2,Lvl,...,0,0,NaN,NaN,NaN,0,6,2006,WD,262500
2031,25,527402250,20,RL,NaN,12537,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,149900


In [5]:
ames.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [6]:
# 2051 Rows, 81 Features
ames.shape

(2051, 81)

In [7]:
# Too Big to Transpose, but left here for reference
ames.describe()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2051.000000,2.051000e+03,2051.000000,1721.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2029.000000,...,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000
mean,1474.033642,7.135900e+08,57.008776,69.055200,10065.208191,6.112140,5.562165,1971.708922,1984.190151,99.695909,...,93.833740,47.556802,22.571916,2.591419,16.511458,2.397855,51.574354,6.219893,2007.775719,181469.701609
std,843.980841,1.886918e+08,42.824223,23.260653,6742.488909,1.426271,1.104497,30.177889,21.036250,174.963129,...,128.549416,66.747241,59.845110,25.229615,57.374204,37.782570,573.393985,2.744736,1.312014,79258.659352
min,1.000000,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,753.500000,5.284581e+08,20.000000,58.000000,7500.000000,5.000000,5.000000,1953.500000,1964.500000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129825.000000
50%,1486.000000,5.354532e+08,50.000000,68.000000,9430.000000,6.000000,5.000000,1974.000000,1993.000000,0.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,162500.000000
75%,2198.000000,9.071801e+08,70.000000,80.000000,11513.500000,7.000000,6.000000,2001.000000,2004.000000,161.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2930.000000,9.241520e+08,190.000000,313.000000,159000.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,1424.000000,547.000000,432.000000,508.000000,490.000000,800.000000,17000.000000,12.000000,2010.000000,611657.000000


## Missing Data

In [8]:
# Fields with missing values sorted in ascending order
ames.isna().sum().sort_values(ascending=False).head()

Pool QC         2042
Misc Feature    1986
Alley           1911
Fence           1651
Fireplace Qu    1000
dtype: int64

### Worst-Offenders
`Pool QC` is in the lead with **2042** nan values, when I look at the data dictionary I see "*PoolQC: Pool Quality,*" then there a couple differnt options for what my possible values could be. These would all make sense but this a huge chunk of our data set! So first I want to check and make sure those features are not mistakes themselves. The other feature to check this against would be `Pool Area`...

In [9]:
ames[ames['Pool Area'] == 0]['Pool Area'].count()

2042

We are in the clear. So it is *relatively* safe to assume that the missing values there are indeed no pool houses. To deal with this we will simply use the `.fillna()` function from Pandas. After looking through the rest of the `na_vals` Series from earlier I came up with the following for loop to take care of the rest of the features.

In [10]:
dirty_cols = ['Pool QC','Misc Feature','Alley','Fence','Fireplace Qu',
             'Garage Finish','Garage Qual','Garage Cond','Garage Type',
             'Bsmt Exposure','BsmtFin Type 2','BsmtFin Type 1','Bsmt Cond',
             'Bsmt Qual','Mas Vnr Type', 'MS SubClass', 'Street']
    
for col in dirty_cols:
    ames[col].fillna('NA',inplace=True)

In [11]:
# looking at missing values again
ames.isna().sum().sort_values(ascending=False).head(12)

Lot Frontage      330
Garage Yr Blt     114
Mas Vnr Area       22
Bsmt Full Bath      2
Bsmt Half Bath      2
Garage Cars         1
Total Bsmt SF       1
Bsmt Unf SF         1
BsmtFin SF 2        1
BsmtFin SF 1        1
Garage Area         1
Kitchen Qual        0
dtype: int64

#### Lot Frontage and Numeric Features

Lot Frontage required a bit of research to deal with. Lot frontage is defined as the horizontal distance between the side lot lines measured along the front lot lines. Basically it is like the depth of a lot. In order to deal with these values, I'm going to take a naive approach and simply assume they are squares, and then take the square of their lot area.

For the rest of the features, I will simply turn the NAs to zeros. A spot check of garage qual nans being equivalent to houses not having garages built leads me to believe we are heading in the right direction

In [12]:
# create a variable to easily work with
ames[ames['Garage Yr Blt'].isna()].groupby('Garage Qual')['Id'].count()[0] == 114

True

In [16]:
ames['Lot Frontage'].fillna(ames['Lot Area']**.5)

0       116.262634
1        43.000000
2        68.000000
3        73.000000
4        82.000000
           ...    
2046     79.000000
2047    111.094554
2048     57.000000
2049     80.000000
2050     60.000000
Name: Lot Frontage, Length: 2051, dtype: float64

In [135]:
# no more na
ames.isna().any().any()

False

## Quality Control

A large amount of features in this data set operate on a scale of NA - Ex. It is a 5 value scale with each raise in value corresponding to a raise in quality. We are going to turn these categorical values into numeric ones. This will give meaning to a score being better or worse than the other to a computer.

In [171]:
# Quality Scaling
cast_dict = {
    'Ex' : 6,
    'Gd' : 5,
    'TA' : 4,
    'Fa' : 3,
    'Po' : 2,
    'NA' : 1
}
# qualities we are looking to scale.
quals = [
    'Kitchen Qual', 'Exter Qual', 'Fireplace Qu', 
    'Bsmt Qual', 'Pool QC', 'Bsmt Cond', 'Garage Qual',
    'Garage Cond', 'Exter Cond'
]
# the loop that did it to em
for qual in quals:
    ames[qual] = ames[qual].map(cast_dict)

In [140]:
# For MS SubClass we are going to take the inverse strategy. We want to assign meaning to a category, not to number. We are also going
# to break down this into smaller more interpretable categories
ames['MS SubClass'] = ames['MS SubClass'].map({
                        20:'One Story',30:'One Story',40:'One Story',45:'One Story',50:'One Story',
                        60:'Two Story',70:'Two Story',75:'Two Story',
                        80:'Split',85:'Split',
                        90:'Duplex',
                        120:'PUD',150:'PUD',160:'PUD',180:'PUD',
                        190:'Conversion'})

# Engineering

## Curb Appeal
The Curb Appeal variable came with the help of Kevin Fernandez. He has years of experience in Real Estate, and offered his consultation to me. Essentially we went through the list of features and tried to pick out ones that **were not numeric scores** and see if we could encapsulate homes having a certain amount of things that may increase property value.

In [160]:
appeal = []

for index in range(ames.shape[0]):
    score = 0
    if (ames['Pool Area'][index] != 0):
        score += 1
    if (ames['Fence'][index] != 'NA') or (ames['Fence'][index] != 'MnWw'):
        score += 1
    if (ames['Garage Type'][index]  == 'CarPort'):
        score += 1
    if (ames['Garage Type'][index]  != 'NA'):
        score += 1
    if (ames['Paved Drive'][index] == 'Y'):
        score += 1
    if (ames['Fireplaces'][index] != 0):
        score += 1
    if (ames['TotRms AbvGrd'][index] > ames['TotRms AbvGrd'].mean()):
        score += 1
    if (ames['Central Air'][index] == 'Y'):
        score += 1
    if (ames['Utilities'][index] == 'AllPub'):
        score += 1
    if (ames['Condition 1'][index] == 'Norm' or ames['Condition 2'][index] == 'Norm'):
        score += 1
    if (ames['Functional'][index] == 'Typ'):
        score += 1
    if (ames['Alley'][index] == 'Pave'):
        score += 1
    if (ames['BsmtFin Type 1'][index] == 'GLQ' or ames['BsmtFin Type 2'][index] == 'GLQ'):
        score += 1
    if (ames['Misc Feature'][index] == 'TenC'):
        score += 1  
    if (ames['Roof Style'][index] == 'Hip'):
        score += 1
    appeal.append(score)
    
ames['curb_appeal'] = appeal

In [143]:
# total sq footage to tell what houses are working with in total area as opposed to being able to focus in on just one
# of the levels
ames['total_SF'] = ames['1st Flr SF'] + ames['2nd Flr SF']
# overalls to get a better idea of the magnitude of the quality of the house
ames['overalls'] = ames['Overall Qual'] * ames['Overall Cond']

## Put it all together in a function(s) 

In [22]:
# shout out: https://stackoverflow.com/a/65663400/16451694
def num_na(df, value):
    '''
        takes a data frame, and value and sets numeric column nas
        to the value.
    '''
    df = df.copy()
    for col in df:
        # select only integer or float dtypes
        if df[col].dtype in ("int", "float"):
            df[col] = df[col].fillna(value)
    return df

In [23]:
def curb_appeal(df):
    '''
        returns the curb appeal score array of a dataframe.
    '''
    appeal = []
    for index in range(df.shape[0]):
        score = 0
        if (df['Pool Area'][index] != 0):
            score += 1
        if (df['Fence'][index] != 'NA') or (df['Fence'][index] != 'MnWw'):
            score += 1
        if (df['Garage Type'][index]  == 'CarPort'):
            score += 1
        if (df['Garage Type'][index]  != 'NA'):
            score += 1
        if (df['Paved Drive'][index] == 'Y'):
            score += 1
        if (df['Fireplaces'][index] != 0):
            score += 1
        if (df['TotRms AbvGrd'][index] > df['TotRms AbvGrd'].mean()):
            score += 1
        if (df['Central Air'][index] == 'Y'):
            score += 1
        if (df['Utilities'][index] == 'AllPub'):
            score += 1
        if (df['Condition 1'][index] == 'Norm' or df['Condition 2'][index] == 'Norm'):
            score += 1
        if (df['Functional'][index] == 'Typ'):
            score += 1
        if (df['Alley'][index] == 'Pave'):
            score += 1
        if (df['BsmtFin Type 1'][index] == 'GLQ' or df['BsmtFin Type 2'][index] == 'GLQ'):
            score += 1
        if (df['Misc Feature'][index] == 'TenC'):
            score += 1  
        if (df['Roof Style'][index] == 'Hip'):
            score += 1
        appeal.append(score)
    return appeal

In [30]:
def clean_dum_ames(df):
    '''
        cleans the data from both data frames, the main purpose is to not have to format the test data every time
    '''
    # Lot Frontage issues
    df['Lot Frontage'].fillna(df['Lot Area']**.5)
    
    df = num_na(df, 0)
    df.fillna('NA',inplace=True)
    
    # NOT COVERED ABOVE THIS GARAGE YEAR HAD A BAD ENTRY, SO I FIXED IT
    df['Garage Yr Blt'] = df['Garage Yr Blt'].apply(lambda x: 2007 if x > 2020 else x)
    
    # curb appeal
    df['curb_appeal'] = curb_appeal(df)
    
    # Quality Scaling
    cast_dict = {
        'Ex' : 6,
        'Gd' : 5,
        'TA' : 4,
        'Fa' : 3,
        'Po' : 2,
        'NA' : 1
    }

    # we take them all except Bsmt Qual, which is measuring height
    quals = [
        'Exter Qual', 'Exter Cond', 'Bsmt Cond', 'Heating QC',
        'Kitchen Qual', 'Fireplace Qu', 'Garage Qual', 'Garage Cond',
        'Pool QC'
    ]
    for qual in quals:
        df[qual] = df[qual].map(cast_dict) 
        
    # NOT MENTIONED ABOVE, THIS WILL ENGINEER A SCORE OF ALL THE QUALTIES NA-EX TOGETHER OF A PROPERTY
    overall_quals = [1] * df.shape[0]
    for qual in quals:
        overall_quals *= df[qual]
    df['overall_quals'] = overall_quals
    
    # mapping the Ms SubClass
    df['MS SubClass'] = df['MS SubClass'].map({20:'One Story',30:'One Story',40:'One Story',45:'One Story',50:'One Story',
                        60:'Two Story',70:'Two Story',75:'Two Story',
                        80:'Split',85:'Split',
                        90:'Duplex',
                        120:'PUD',150:'PUD',160:'PUD',180:'PUD',
                        190:'Conversion'})
    
    df['total_SF'] = df['1st Flr SF'] + df['2nd Flr SF']
    df['overalls'] = df['Overall Qual'] * df['Overall Cond']
    # age of house at purchase
    df['house_age'] = df['Yr Sold'] - df['Year Built']
    # recession
    df['recession'] = df['Yr Sold'].apply(lambda x: 'Yes' if x >= 2008 else 'No' )
    # dummying 
    cols_to_dummy = df.select_dtypes(include=['object','category']).columns
    
    return pd.get_dummies(df,drop_first=True,columns=cols_to_dummy)

## Saving the dataframes

In [31]:
training_df = clean_dum_ames(pd.read_csv('./data/train.csv'))
testing_df = clean_dum_ames(pd.read_csv('./data/test.csv'))
training_df.to_csv('./frames/training.csv')
testing_df.to_csv('./frames/testing.csv')